In [52]:
import pandas as pd
from SQLCode import DatabaseConnection
from SQLCode import DatabaseCredentials as DBC
import numpy as np


from sklearn.feature_extraction.text import CountVectorizer

from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

from sklearn import preprocessing

In [55]:
# Opening connection
creds = DBC.DataBaseCredentials()
conn = DatabaseConnection.sql_connection(creds.server, creds.database, creds.user, creds.password)
connection = conn.open()
cursor = connection.cursor()

In [56]:
# liveFeed = pd.read_sql_query("select * from live_feed", connection)
liveFeed = pd.read_sql_query("select * from live_feed where gameID = 2020020634", connection)

In [57]:
# Getting the seasons data
seasons = pd.read_sql_query("select * from schedules", connection)

In [58]:
# boxscores = pd.read_sql_query("select gameID, teamID, playerID from box_scores", connection)
boxscores = pd.read_sql_query("select gameID, teamID, playerID from box_scores where gameID = 2020020634", connection)

# Creating Model Input

In [59]:
# Filtering to regular seasons games and 20102011 onwards (when live data started)
seasonsFiltered = seasons[seasons['seasonID'] >= 20102011]
seasonsFiltered = seasonsFiltered[seasonsFiltered['gameType'] == 'R']

In [60]:
# Getting and filtering the raw data
rawData = pd.merge(liveFeed,seasons, how='inner',on='gameID')
rawData = rawData[rawData['seasonID'] >= 20102011]
# rawData = rawData[rawData['gameType'] == 'R']
# Keeping assists so we can add them in
rawData = rawData[(rawData['eventSubID'] == 0) | (rawData['playerType'] == 'Assist')]
rawData = rawData[['eventID',
         'eventSubID',
         'gameID',
         'eventTypeID',
         'playerType',
         'eventDescription',
         'periodNum',
         'periodTime',
         'xCoordinate',
         'yCoordinate',
         'teamID',
         'homeTeamID','awayTeamID','playerID','penaltyMinutes']]

In [61]:
# Adding in assist
rawData['eventTypeID'] = np.where((rawData['eventTypeID'] == 'GOAL') & (rawData['playerType'] == 'Assist'), 'ASSIST', rawData['eventTypeID'])

In [62]:
# Dont need this column anymore
rawData = rawData.drop('playerType',axis=1)

In [63]:
# Merging box score data
rawData = pd.merge(rawData, 
                   boxscores, 
                   how='left', 
                   left_on=['gameID', 'playerID'], 
                   right_on=['gameID', 'playerID'],
                   suffixes=('', '_box'))

In [64]:
# Getting only the desired events
rawDataFiltered = rawData[rawData['eventTypeID'].isin(['FACEOFF',
                             'SHOT',
                             'MISSED_SHOT',
                             'BLOCKED_SHOT',
                             'TAKEAWAY',
                             'GIVEAWAY',
                             'HIT',
                             'GOAL',
                             'ASSIST',
                             'PERIOD_END',
                             'EARLY_INT_START',
                             'PENALTY',
                             'STOP',
                             'EARLY_INT_END',
                             'EARLY_INT_END'])]
# rawDataFiltered = rawDataFiltered[rawDataFiltered['playerType'].isin(['NULL', 
#                                    'Winner', 
#                                    'Loser', 
#                                    'Hitter', 
#                                    'PlayerID','Shooter','Blocker','Unknown','Scorer','PenaltyOn','DrewBy'])]


# Sorting
rawDataFiltered = rawDataFiltered.sort_values(by=['gameID', 'eventID','eventSubID'],ascending=[True,True,False])

In [65]:
# Don't need this column anymore
rawDataFiltered = rawDataFiltered.drop('eventSubID',axis=1)

In [66]:
# Converting period number to numeric (seems to be read in as a string)
rawDataFiltered['periodNum'] = pd.to_numeric(rawDataFiltered['periodNum'])

# Filtering to only the 1st threee period for consistency
rawDataFiltered = rawDataFiltered[rawDataFiltered['periodNum'] <= 3]

In [67]:
def team_type(teamID, homeTeamID, awayTeamID):
    if teamID == homeTeamID:
        return 'HOME'
    elif teamID == awayTeamID:
        return 'AWAY'
    else:
        return np.nan

def coordinate_normalization(xCoord, teamType, periodNum):
    if teamType == np.nan:
        return None
    else:
        if int(periodNum)%2 == 1:
            if teamType == 'AWAY':
                return xCoord
            else:
                return -1 * xCoord
        else:
            if teamType == 'AWAY':
                return -1 * xCoord
            else:
                return xCoord
            
def elapsed_seconds(periodNum, periodTime):
    periodTime = pd.Timedelta(periodTime)
    return (int(periodNum) - 1) * 20 * 60 + periodTime.total_seconds()/60

class Queue:
    #CITATION: https://runestone.academy/runestone/books/published/pythonds/BasicDS/ImplementingaQueueinPython.html
    def __init__(self):
        self.queue = []

    def isEmpty(self):
        return self.queue == []

    def enqueue(self, item):
        self.queue.insert(0,item)

    def dequeue(self):
        return self.queue.pop()

    def size(self):
        return len(self.queue)
    
    def get_queue(self):
        return self.queue

    def exchange(self, oldItem, newItem):
        self.queue[self.queue.index(oldItem)] = newItem
        
    def remove(self, item):
        self.queue.remove(item)

In [68]:
rawDataFiltered['teamType'] = rawDataFiltered.apply(lambda row: team_type(row['teamID'], 
                                                                          row['homeTeamID'], 
                                                                          row['awayTeamID']) ,axis=1)

In [69]:
rawDataFiltered['xCoordinate'] = rawDataFiltered.apply(lambda row: coordinate_normalization(row['xCoordinate'], 
                                                           row['teamType'], 
                                                           row['periodNum']), axis=1)

In [70]:
rawDataFiltered['secondsElapsed'] = rawDataFiltered.apply(lambda row: elapsed_seconds(row['periodNum'], row['periodTime']), axis=1)

In [71]:
rawDataFiltered[rawDataFiltered['eventTypeID'] == 'ASSIST']

,eventID,gameID,eventTypeID,eventDescription,periodNum,periodTime,xCoordinate,yCoordinate,teamID,homeTeamID,awayTeamID,playerID,penaltyMinutes,teamID_box,teamType,secondsElapsed
30,28,2020020634,ASSIST,"Matthew Tkachuk (15) Wrist Shot, assists: John...",1,0 days 07:41:00,81.0,-1.0,20.0,20,23,8479976.0,NaN,20.0,HOME,461.0
29,28,2020020634,ASSIST,"Matthew Tkachuk (15) Wrist Shot, assists: John...",1,0 days 07:41:00,81.0,-1.0,20.0,20,23,8476346.0,NaN,20.0,HOME,461.0
58,54,2020020634,ASSIST,"Dillon Dube (11) Backhand, assists: Derek Ryan...",1,0 days 12:16:00,83.0,1.0,20.0,20,23,8476439.0,NaN,20.0,HOME,736.0
57,54,2020020634,ASSIST,"Dillon Dube (11) Backhand, assists: Derek Ryan...",1,0 days 12:16:00,83.0,1.0,20.0,20,23,8478585.0,NaN,20.0,HOME,736.0
81,75,2020020634,ASSIST,"Andrew Mangiapane (18) Wrist Shot, assists: Mi...",1,0 days 16:18:00,85.0,2.0,20.0,20,23,8475807.0,NaN,20.0,HOME,978.0
80,75,2020020634,ASSIST,"Andrew Mangiapane (18) Wrist Shot, assists: Mi...",1,0 days 16:18:00,85.0,2.0,20.0,20,23,8474150.0,NaN,20.0,HOME,978.0
119,111,2020020634,ASSIST,"Brett Ritchie (4) Wrist Shot, assists: Derek R...",2,0 days 05:14:00,64.0,-3.0,20.0,20,23,8479346.0,NaN,20.0,HOME,1514.0
118,111,2020020634,ASSIST,"Brett Ritchie (4) Wrist Shot, assists: Derek R...",2,0 days 05:14:00,64.0,-3.0,20.0,20,23,8478585.0,NaN,20.0,HOME,1514.0
131,121,2020020634,ASSIST,"Bo Horvat (19) Snap Shot, assists: Brock Boese...",2,0 days 08:34:00,67.0,2.0,23.0,20,23,8480056.0,NaN,23.0,AWAY,1714.0
130,121,2020020634,ASSIST,"Bo Horvat (19) Snap Shot, assists: Brock Boese...",2,0 days 08:34:00,67.0,2.0,23.0,20,23,8478444.0,NaN,23.0,AWAY,1714.0


In [72]:
sequences = []
actionEvents = ['FACEOFF',
                'SHOT',
                'MISSED_SHOT',
                'BLOCKED_SHOT',
                'TAKEAWAY',
                'GIVEAWAY',
                'HIT',
                'ASSIST',
                'GOAL']
startEndEvents = ['PERIOD_START',
                  'PERIOD_END',
                  'EARLY_INT_START',
                  'PENALTY',
                  'STOP',
                  'SHOOTOUT_COMPLETE',
                  'GAME_END',
                  'EARLY_INT_END',
                  'EARLY_INT_END']

sequenceNum = 0
eventNum = 0
penaltyQueue = Queue()
gameID = 0
context = {'goalDiff':0, 'manpowerDiff':0, 'periodNum':1}
for index, row in rawDataFiltered.iterrows():
    if index % 100000 == 0:
        print((index/len(rawDataFiltered))*100, '%')
    
    # Resetting the context
    if gameID != row['gameID']:
        gameID = row['gameID']
        context = {'goalDiff':0, 'manpowerDiff':0, 'periodNum':1}
        
    # Updating the context if needed
    if row['periodNum'] != context['periodNum']:
        context['periodNum'] = row['periodNum']
        
    # Computing if respective team is home or away
    teamType = team_type(row['teamID'], row['homeTeamID'], row['awayTeamID'])
    
     # If there currently is a penalty in the penalty queue
    if penaltyQueue.size() > 0:
        # Determining the time of the action
        actionTime = elapsed_seconds(row['periodNum'],row['periodTime'])

        # Iterating through all  the penalties in the queue from oldest to newest
        for penalty in reversed(penaltyQueue.get_queue()):
            # If the action occured after the penalty ended, we update the context and pop the penalty
            if penalty['penaltyEnd'] < actionTime:
                # Popping the penalty
                penaltyQueue.remove(penalty)

                # Updating the context
                if penalty['team'] == 'HOME':
                    context['manpowerDiff'] -= 1
                else:
                    context['manpowerDiff'] -= -1
        
    # Catching penalties to update the context
    if row['eventTypeID'] == 'PENALTY':
        if int(row['penaltyMinutes']) != 10:
            # Getting the end time of the penalty (in seconds)
            penaltyStart = elapsed_seconds(row['periodNum'],row['periodTime'])
            penaltyEnd = penaltyStart + row['penaltyMinutes'] * 60
            

            # Determining who took the penalty to update the context
            if teamType == 'HOME':
                context['manpowerDiff'] += 1
            else:
                context['manpowerDiff'] += -1

            # Enqueuing the penalty
            penaltyQueue.enqueue({'team':teamType, 
                                  'penaltyStart': penaltyStart, 
                                  'penaltyEnd':penaltyEnd, 
                                  'penaltyLength': 
                                  row['penaltyMinutes']})
            
    # Adding in the event if it is a goal or assist (doing it here in since the context will be updated below)
    if (row['eventTypeID'] == 'ASSIST'):
        sequences.append([row['gameID'],  
                          context['goalDiff'],
                          context['manpowerDiff'],
                          context['periodNum'],
                          sequenceNum, 
                          eventNum, 
                          row['eventTypeID'],  # action type
                          teamType,  # Home/Away
                          row['secondsElapsed'],
                          row['xCoordinate'],
                          row['yCoordinate'],
                          row['playerID']]) #Neutral/Offensive/Defensive/NULL

           
    # If the event type is a goal
    if row['eventTypeID'] == 'GOAL':

        # Injecting the shot before the goal# Adding in the next sequency
        sequences.append([row['gameID'],  
                  context['goalDiff'],
                  context['manpowerDiff'],
                  context['periodNum'],
                  sequenceNum, 
                  eventNum, 
                  'SHOT',  # action type
                  teamType,  # Home/Away
#                       row['zone']
                  row['secondsElapsed'],
                      row['xCoordinate'],
                      row['yCoordinate'],
                  row['playerID']]) #Neutral/Offensive/Defensive/NULL
        # Incrementing the event number
        eventNum += 1
        
        # Adding in the goal
        sequences.append([row['gameID'],  
                      context['goalDiff'],
                      context['manpowerDiff'],
                      context['periodNum'],
                      sequenceNum, 
                      eventNum, 
                      row['eventTypeID'],  # action type
                      teamType,  # Home/Away
                      row['secondsElapsed'],
                      row['xCoordinate'],
                      row['yCoordinate'],
                      row['playerID']]) #Neutral/Offensive/Defensive/NULL

        # Updating the context
        if teamType == 'HOME':
            context['goalDiff'] -= 1
        else:
            context['goalDiff'] += 1

        # Defining home/away flags to only pop off the minimum number of penalties
        FLAGS = {'HOME':True, 'AWAY':True}

        # Determining the time of the action
        actionTime = elapsed_seconds(row['periodNum'],row['periodTime'])

        # If there currently is a penalty in the penalty queue
        if penaltyQueue.size() > 0:

            # Iterating through all  the penalties in the queue from oldest to newest
            for penalty in reversed(penaltyQueue.get_queue()):

                # If the penalty is a 5 minute major, the player must serve the full 5 minutes (no change needed)
                # If the penalty is over it would have been popped in the above if statement
                if penalty['penaltyLength'] == 5:
                    continue
                else:

                    # Making sure its not a shorthanded goal and we haven't already popped a penalty for this goal/team
                    if (penalty['team'] != teamType) & (FLAGS[penalty['team']]):

                        # Creating the updated penalty
                        newPenalty = penalty
                        newPenalty['penaltyStart'] = actionTime
                        newPenalty['penaltyLength'] += -120
                        newPenalty['penaltyEnd'] = newPenalty['penaltyStart'] + newPenalty['penaltyLength']

                        if penalty['penaltyLength'] <= 0:

                            # Popping the penalty off
                            penaltyQueue.remove(penalty)

                            # Updating the context
                            if penalty['team'] == 'HOME':
                                context['manpowerDiff'] -= 1
                            else:
                                context['manpowerDiff'] -= -1
                        else:

                            # replacing the old penalty info with the new one
                            penaltyQueue.exchange(penalty, newPenalty)

                        FLAGS[penalty['team']] = False 
                            
#     # Updating the context if needed
#     if row['periodNum'] != context['periodNum']:
#         context['periodNum'] = row['periodNum']
    
    # Adding in the next sequency
    if (((row['eventTypeID'] not in startEndEvents) | 
         (row['eventTypeID'] == 'PENALTY')) & 
        (row['eventTypeID'] != 'GOAL') & 
        (row['eventTypeID'] != 'ASSIST')):
        sequences.append([row['gameID'],  
                          context['goalDiff'],
                          context['manpowerDiff'],
                          context['periodNum'],
                          sequenceNum, 
                          eventNum, 
                          row['eventTypeID'],  # action type
                          teamType,  # Home/Away
                          row['secondsElapsed'],
                          row['xCoordinate'],
                          row['yCoordinate'],
                          row['playerID']]) #Neutral/Offensive/Defensive/NULL
    if row['eventTypeID'] in actionEvents:
        eventNum += 1
    else:
        sequenceNum += 1
        eventNum = 0
#     break
sequenceData = pd.DataFrame(sequences, 
                            columns=['gameID', 
                                     'goalDiff', 
                                     'manpowerDiff',
                                     'periodNum', 
                                     'sequenceNum',
                                     'eventNum',
                                     'event', 
                                     'team',
                                     'secondsElapsed',
                                     'xCoord',
                                     'yCoord',
                                     'playerID'])

In [73]:
# Removing any rows that contain a null value
sequenceData = sequenceData[~sequenceData['sequenceNum'].isin(sequenceData[sequenceData.isnull().any(axis=1)]['sequenceNum'].values)]

In [74]:
# Vectorizing the events
eventVectorizer = CountVectorizer()
eventVectorizer.fit(sequenceData['event'])
vectorizedEvents = eventVectorizer.transform(sequenceData['event']).toarray()

# List of actions in the correct order according to its position on the vectorized array
actions = [action[0] for action in sorted(eventVectorizer.vocabulary_.items())]

In [75]:
# Adding in the vectorized columns
for action in range(0,len(actions)):
    sequenceData.insert(column=actions[action], value=vectorizedEvents[:,action], loc=len(sequenceData.columns))

In [76]:
# # List of actions in the correct order according to its position on the vectorized array
# actions = [action[0] for action in sorted(eventVectorizer.vocabulary_.items())]

# # Adding the vectorized events onto the sequence data
# sequenceData = pd.concat([sequenceData,
#                           pd.DataFrame(data=vectorizedEvents,
#                                        columns=actions)],axis=1)

# Dropping event as it is no longer needed
sequenceData = sequenceData.drop(['event'],axis=1)

In [77]:
# Filling NAs
sequenceData['team'] = sequenceData['team'].fillna(value='neither')

In [78]:
# Vectorizing the teams
teamVectorizer = CountVectorizer()
teamVectorizer.fit(sequenceData['team'])
vectorizedTeams = teamVectorizer.transform(sequenceData['team']).toarray()

# List of teams in the correct order according to its position on the vectorized array
teams = [action[0] for action in sorted(teamVectorizer.vocabulary_.items())]


In [79]:
# Adding in the vectorized columns
for team in range(0,len(teams)):
    sequenceData.insert(column=teams[team], value=vectorizedTeams[:,team], loc=len(sequenceData.columns))

In [80]:
# # List of teams in the correct order according to its position on the vectorized array
# teams = [action[0] for action in sorted(teamVectorizer.vocabulary_.items())]

# Adding the vectorized events onto the sequence data
# sequenceData = pd.concat([sequenceData,
#                           pd.DataFrame(data=vectorizedTeams,
#                                        columns=teams)],axis=1)

# Dropping event as it is no longer needed
sequenceData = sequenceData.drop(['team'],axis=1)

In [81]:
colsTransformed = ['goalDiff', 
              'manpowerDiff',
          'periodNum',
          'secondsElapsed', 
          'xCoord', 
          'yCoord']

In [82]:
scaler = preprocessing.StandardScaler().fit(sequenceData[colsTransformed].values)

In [83]:
scaledData = scaler.transform(sequenceData[colsTransformed].values)

In [84]:
for i in range(0,len(colsTransformed)):
    sequenceData[colsTransformed[i]] = scaledData[:,i]

In [85]:
# Defining a sequence data object
sequenceDataTMinusOne = sequenceData.drop(['goalDiff', 'manpowerDiff', 'periodNum'],axis=1).copy(deep=True)

# Incrementing the eventnumber to use to join below
sequenceDataTMinusOne['eventNum'] -= 1

In [86]:
# Merging the time t and t+1 datasets
sequenceDataComplete = pd.merge(left=sequenceData, 
         right=sequenceDataTMinusOne, 
         how='left',
         left_on=['gameID', 'sequenceNum', 'eventNum'],
         right_on=['gameID', 'sequenceNum', 'eventNum'],
        suffixes=('','_TMinusOne'))

In [87]:
columns = ['gameID', 
          'sequenceNum',
          'eventNum', 
          'goalDiff', 
          'manpowerDiff',
          'periodNum',
          'secondsElapsed', 
          'xCoord', 
          'yCoord', 
          'blocked_shot',
          'faceoff', 
          'giveaway', 
          'goal', 
          'assist',
          'hit',
          'missed_shot',
          'penalty', 
          'shot',
          'takeaway',
          'away', 
          'home',
          'playerID',
          'secondsElapsed_TMinusOne',
          'xCoord_TMinusOne',
          'yCoord_TMinusOne', 
          'blocked_shot_TMinusOne',
          'faceoff_TMinusOne', 
          'giveaway_TMinusOne', 
          'goal_TMinusOne',
          'assist_TMinusOne',
          'hit_TMinusOne', 
          'missed_shot_TMinusOne',
          'penalty_TMinusOne',
          'shot_TMinusOne', 
          'takeaway_TMinusOne', 
          'away_TMinusOne',
          'home_TMinusOne']

In [88]:
# Adding in any missing columns (setting them to be 0)
for col in columns:
    if col not in sequenceDataComplete.columns:
        sequenceDataComplete[col] = np.zeros((len(sequenceDataComplete)))

In [89]:
# Re-ordering columns
sequenceDataComplete = sequenceDataComplete[columns]

In [90]:
sequenceDataComplete.columns

Index(['gameID', 'sequenceNum', 'eventNum', 'goalDiff', 'manpowerDiff',
       'periodNum', 'secondsElapsed', 'xCoord', 'yCoord', 'blocked_shot',
       'faceoff', 'giveaway', 'goal', 'assist', 'hit', 'missed_shot',
       'penalty', 'shot', 'takeaway', 'away', 'home', 'playerID',
       'secondsElapsed_TMinusOne', 'xCoord_TMinusOne', 'yCoord_TMinusOne',
       'blocked_shot_TMinusOne', 'faceoff_TMinusOne', 'giveaway_TMinusOne',
       'goal_TMinusOne', 'assist_TMinusOne', 'hit_TMinusOne',
       'missed_shot_TMinusOne', 'penalty_TMinusOne', 'shot_TMinusOne',
       'takeaway_TMinusOne', 'away_TMinusOne', 'home_TMinusOne'],
      dtype='object')

# Model Creation
## Model Definition

In [91]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    data = torch.from_numpy(df.values)
    data = data.type(torch.float64)
    data = data.to(device)
    return data

def custom_loss(modelInput, model, i):
    t = modelInput[0:i+1,:,3:21]
    tPlusOne = modelInput[0:i+1,:,[3,4,5,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36]]
    
    # If goal scored by home team (if event is a goal, and team is home)
    if (modelInput[i,:,[12]] == 1) & (modelInput[i,:,[20]] == 1):
        R = torch.tensor([1,0,0]).to(get_device())
        
    # If goal scored by away team (if event is a goal, and team is away) 
    elif (modelInput[i,:,[12]] == 1) & (modelInput[i,:,[19]] == 1):
        R = torch.tensor([0,1,0]).to(get_device())
        
    # No one scored
    else:
        R = torch.tensor([0,0,1]).to(get_device())
    
    # If there was no previous action
#     if torch.sum(torch.isnan(tPlusOne)) > 0:
#         loss = R + model(t)
#     else: 
#         loss = R + model(tPlusOne) - model(t) 
        
    if torch.sum(torch.isnan(tPlusOne)) > 0:
        loss = R - model(t)
    else: 
        loss = R + model(tPlusOne) - model(t) 
    
    
    return torch.mean(torch.square(loss))

In [92]:
class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.inputSize = 18
        self.numLSTMNodes = 1000
        self.numLSTMLayers = 1
        
        self.lstmLayer = nn.LSTM(input_size=self.inputSize, 
                                  hidden_size=self.numLSTMNodes,
                                  num_layers=self.numLSTMLayers, 
                                  bias=True, 
                                  dropout=0, 
                                  batch_first=True).double()
        self.hidden1 = nn.Linear(in_features=self.numLSTMNodes, out_features=1000).double()   
        self.hidden2 = nn.Linear(in_features=1000, out_features=1000).double()   
        self.hidden3 = nn.Linear(in_features=1000, out_features=1000).double()   
        self.hidden4 = nn.Linear(in_features=1000, out_features=1000).double()
        self.output = nn.Linear(in_features=1000, out_features=3).double()
        
    def forward(self, modelInput):
        hidden = (
                    torch.cuda.FloatTensor(self.numLSTMLayers , 1, self.numLSTMNodes).normal_().double() ,
                    torch.cuda.FloatTensor(self.numLSTMLayers , 1, self.numLSTMNodes).normal_().double() 
                  )
        for sequence in modelInput:
            out, hidden = self.lstmLayer(sequence.view(1,1,-1), hidden)
        t = F.relu(out)
        t = F.relu(self.hidden1(t))
        t = F.relu(self.hidden2(t))
        t = F.relu(self.hidden3(t))
        t = F.relu(self.hidden4(t))
        t = F.softmax(self.output(t),dim=2)
        return t   

In [93]:
# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    data = torch.from_numpy(df.values)
    data = data.type(torch.float64)
    data = data.to(device)
    return data

In [94]:
# Moving the df to GPU
dfGPU = df_to_tensor(sequenceDataComplete)

In [95]:
# Reshaping
# (batch size, sequence length, num features)
dfGPU = dfGPU.reshape((-1,1,37))

## Model Training

In [ ]:
# network = DQN()
# device = get_device()
# network.to(device)

# network = network.type(torch.float64)

# optimizer = optim.Adam(network.parameters(), lr=0.0001)
    
# # Each Episode
# for gameID in sequenceDataComplete['gameID'].unique():
#     print(gameID)
#     gameDataGPU = dfGPU[dfGPU[:,:,0][:,0] == gameID,:,:]
#     gameData = sequenceDataComplete[sequenceDataComplete['gameID'] == gameID]
#     for sequenceNum in gameData['sequenceNum'].unique():

       
#         modelInput = gameDataGPU[gameDataGPU[:,:,1][:,0] == sequenceNum,:,:]
            
#         i = 0
#         while i <  modelInput.shape[0]:
#             # calculate the loss
#             loss = custom_loss(modelInput, network, i)
        
#             i += 1

# #             if sequenceNum > 0:
# #                 break
# #             print('**********************')


#             # zero gradients
#             optimizer.zero_grad()

#             # perform backprop and update weights
#             loss.backward()
#             optimizer.step()

        

        

In [97]:
# Loading model
network = DQN()
network.load_state_dict(torch.load('gim_model.pt'))
network.eval()
network.to(get_device())

DQN(
  (lstmLayer): LSTM(18, 1000, batch_first=True)
  (hidden1): Linear(in_features=1000, out_features=1000, bias=True)
  (hidden2): Linear(in_features=1000, out_features=1000, bias=True)
  (hidden3): Linear(in_features=1000, out_features=1000, bias=True)
  (hidden4): Linear(in_features=1000, out_features=1000, bias=True)
  (output): Linear(in_features=1000, out_features=3, bias=True)
)

## Compiling Results

In [111]:
GIM = []
# Each Episode
for gameID in sequenceDataComplete['gameID'].unique():
    print(gameID)
    gameDataGPU = dfGPU[dfGPU[:,:,0][:,0] == gameID,:,:]
    gameData = sequenceDataComplete[sequenceDataComplete['gameID'] == gameID]
    for sequenceNum in gameData['sequenceNum'].unique():
        modelInput = gameDataGPU[gameDataGPU[:,:,1][:,0] == sequenceNum,:,:]
        i = 0
        while i <  modelInput.shape[0]:
            if  i == 0:
                Q_t = network(modelInput[0:i+1,:,3:21])
                gim_t = Q_t
#                 print(modelInput[0:i+1,:,3:20])
#                 print(Q_t)
            else:
                Q_tMinusOne = Q_t
                Q_t = network(modelInput[0:i+1,:,3:21])
#                 gim_t = Q_t - Q_tMinusOne
                gim_t = Q_t
            GIM.append([int(modelInput[i,0,0].item()),# GameID
                        int(modelInput[i,0,1].item()),# sequenceNumber
                        int(modelInput[i,0,2].item()),# eventNumber
                        modelInput[-1,0,21].item(), #PlayerID
                        modelInput[-1,0,19].item(), # Away
                        modelInput[-1,0,20].item(), # Home
                        gim_t[0,0,0].item(), # Home Probability
                        gim_t[0,0,1].item(), # Away Probability
                        gim_t[0,0,2].item()]) # Neither Probability
            i += 1
            
#             break
#         break
#     break

       
results = pd.DataFrame(GIM, columns=['gameID', 
                                     'sequenceNum',
                                     'eventNum',
                                     'playerID', 
                                     'awayTeam',
                                     'homeTeam', 
                                     'homeProbability',
                                     'awayProbability',
                                     'neitherProbability'])

2020020634
torch.Size([1, 1, 37]) 0
Sequence Num: 0.0 Event Num 0.0
**********************
torch.Size([9, 1, 37]) 0
Sequence Num: 4.0 Event Num 0.0
**********************
torch.Size([9, 1, 37]) 1
Sequence Num: 4.0 Event Num 1.0
**********************
torch.Size([9, 1, 37]) 2
Sequence Num: 4.0 Event Num 2.0
**********************
torch.Size([9, 1, 37]) 3
Sequence Num: 4.0 Event Num 3.0
**********************
torch.Size([9, 1, 37]) 4
Sequence Num: 4.0 Event Num 4.0
**********************
torch.Size([9, 1, 37]) 5
Sequence Num: 4.0 Event Num 5.0
**********************
torch.Size([9, 1, 37]) 6
Sequence Num: 4.0 Event Num 6.0
**********************
torch.Size([9, 1, 37]) 7
Sequence Num: 4.0 Event Num 7.0
**********************
torch.Size([9, 1, 37]) 8
Sequence Num: 4.0 Event Num 8.0
**********************
torch.Size([5, 1, 37]) 0
Sequence Num: 5.0 Event Num 0.0
**********************
torch.Size([5, 1, 37]) 1
Sequence Num: 5.0 Event Num 1.0
**********************
torch.Size([5, 1, 37]) 2
Sequ

In [ ]:
torch.save(network.state_dict(),'model_all_games.pt')

In [ ]:
results['value'] = np.where(results['awayTeam'] == 1, results['awayProbability'], results['homeProbability'])

In [ ]:
results.to_csv('deep_rl_results_all_games.csv')

In [ ]:
resultsComplete = pd.merge(results, seasons[['seasonID','gameID']], left_on=['gameID'],right_on=['gameID'])

In [ ]:
resultsComplete = resultsComplete[['seasonID','playerID','value']].groupby(['seasonID','playerID']).sum('value').reset_index().sort_values('value',ascending=False)

In [ ]:
players = pd.read_sql_query("select playerID, firstName, lastName from players", connection)

In [ ]:
pd.merge(resultsComplete, players,on='playerID').sort_values('value',ascending=False).to_csv('deep_rl_results_all_games_aggregated.csv')